In [1]:
import nltk
import time
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

import csv
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize,blankline_tokenize
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import string
from collections import Counter
import json
import re as regex
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import numpy as np

In [2]:
import findspark
findspark.init()
#findspark.init("C:\opt\spark\spark-2.3.0-bin-hadoop2.7")
import pyspark.sql.types as typ
from pyspark.sql import SparkSession

In [3]:
sparkSession = SparkSession.builder \
       .master("local") \
       .appName("Spark ML") \
       .getOrCreate()

In [4]:
#read json file into Spark DataFrame
#create spark dataframe from Pandas DF
df=pd.read_json("data/Microblog_Trialdata.json")
def list_sp(row):
    idx=row["spans"]
    #ch="".join(x for x in idx if x)
    ch=' '.join(idx)
    row["spans"]=ch
    return row
df = df.apply(list_sp, axis=1)
train_data=sparkSession.createDataFrame(df)
df.head()

,cashtag,id,sentiment score,source,spans
0,$F,5540055,-0.454,stocktwits,Putting on a little $F short
1,$AAPL,10752226,-0.464,stocktwits,short some
2,$BAC,10920221,0.445,stocktwits,buying opportunity
3,$SHOR,12971398,0.661,stocktwits,Scaling Up on Long Position
4,$JPM,16142438,-0.763,stocktwits,its time to sell banks


In [5]:
#extraire les champ de sentiment score et tweets
import pyspark.mllib.regression
from pyspark.mllib.regression import LabeledPoint
train_data.show()
spans=train_data.select("sentiment score","spans")



+-------+------------------+---------------+----------+--------------------+
|cashtag|                id|sentiment score|    source|               spans|
+-------+------------------+---------------+----------+--------------------+
|     $F|           5540055|         -0.454|stocktwits|Putting on a litt...|
|  $AAPL|          10752226|         -0.464|stocktwits|          short some|
|   $BAC|          10920221|          0.445|stocktwits|  buying opportunity|
|  $SHOR|          12971398|          0.661|stocktwits|Scaling Up on Lon...|
|   $JPM|          16142438|         -0.763|stocktwits|its time to sell ...|
|   $LMT|          14073133|          0.627|stocktwits|       Entering long|
|   $DNN|          18479024|          0.653|stocktwits|      picked some up|
|   $CRK|          34147106|          0.668|stocktwits|time to accumulat...|
|   $CRK|          34147107|           0.46|stocktwits|Looking for a str...|
|   $CRK|          34147108|          0.403|stocktwits|Very intrigued wi...|

In [6]:
#renommer le champ sentiment score en "label"
spans=spans.toDF("label","spans")
#temp = new.rdd.map(lambda line:LabeledPoint(line[0],[line[1:]]))


In [7]:
spans.show(truncate=False)

+------+--------------------------------------------------------------------+
|label |spans                                                               |
+------+--------------------------------------------------------------------+
|-0.454|Putting on a little $F short                                        |
|-0.464|short some                                                          |
|0.445 |buying opportunity                                                  |
|0.661 |Scaling Up on Long Position                                         |
|-0.763|its time to sell banks                                              |
|0.627 |Entering long                                                       |
|0.653 |picked some up                                                      |
|0.668 |time to accumulate for a long position far more upside than downside|
|0.46  |Looking for a strong bounce Lunchtime rally coming                  |
|0.403 |Very intrigued with the technology and growth potential 

In [8]:
#remove some additional features(numbers and unctuatuion)
import pyspark.ml.feature as ft
tokenizer = ft.RegexTokenizer(
inputCol='spans',
outputCol='tokens',
pattern='\s+|[$,.\"]')

In [9]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
countTokens = udf(lambda words: len(words), IntegerType())
tok = tokenizer \
    .transform(spans) \


In [10]:
tok.show()

+------+--------------------+--------------------+
| label|               spans|              tokens|
+------+--------------------+--------------------+
|-0.454|Putting on a litt...|[putting, on, a, ...|
|-0.464|          short some|       [short, some]|
| 0.445|  buying opportunity|[buying, opportun...|
| 0.661|Scaling Up on Lon...|[scaling, up, on,...|
|-0.763|its time to sell ...|[its, time, to, s...|
| 0.627|       Entering long|    [entering, long]|
| 0.653|      picked some up|  [picked, some, up]|
| 0.668|time to accumulat...|[time, to, accumu...|
|  0.46|Looking for a str...|[looking, for, a,...|
| 0.403|Very intrigued wi...|[very, intrigued,...|
|   0.0|short2 48 + - ***...|[short2, 48, +, -...|
|-0.438|Biggest Market Lo...|[biggest, market,...|
|-0.398|$GOOG $GOOGL woul...|[goog, googl, wou...|
| 0.483| Buying $SBUX on dip|[buying, sbux, on...|
| -0.48|is a short below ...|[is, a, short, be...|
|-0.454|don't Putting on ...|[don't, putting, ...|
+------+--------------------+--

In [11]:
#now remove stopwords from the review(list of words)    
from pyspark.ml.feature import StopWordsRemover
    
remover=StopWordsRemover(inputCol="tokens", outputCol="filtered")
filtered_df=remover.transform(tok)


In [12]:
filtered_df.show()

+------+--------------------+--------------------+--------------------+
| label|               spans|              tokens|            filtered|
+------+--------------------+--------------------+--------------------+
|-0.454|Putting on a litt...|[putting, on, a, ...|[putting, little,...|
|-0.464|          short some|       [short, some]|             [short]|
| 0.445|  buying opportunity|[buying, opportun...|[buying, opportun...|
| 0.661|Scaling Up on Lon...|[scaling, up, on,...|[scaling, long, p...|
|-0.763|its time to sell ...|[its, time, to, s...| [time, sell, banks]|
| 0.627|       Entering long|    [entering, long]|    [entering, long]|
| 0.653|      picked some up|  [picked, some, up]|            [picked]|
| 0.668|time to accumulat...|[time, to, accumu...|[time, accumulate...|
|  0.46|Looking for a str...|[looking, for, a,...|[looking, strong,...|
| 0.403|Very intrigued wi...|[very, intrigued,...|[intrigued, techn...|
|   0.0|short2 48 + - ***...|[short2, 48, +, -...|[short2, 48, +

In [13]:
#now make 2-gram model
from pyspark.ml.feature import NGram

ngram=NGram(n=1, inputCol="filtered", outputCol="n-gram")
gram_df=ngram.transform(filtered_df)

In [14]:
gram_df.show()

+------+--------------------+--------------------+--------------------+--------------------+
| label|               spans|              tokens|            filtered|              n-gram|
+------+--------------------+--------------------+--------------------+--------------------+
|-0.454|Putting on a litt...|[putting, on, a, ...|[putting, little,...|[putting, little,...|
|-0.464|          short some|       [short, some]|             [short]|             [short]|
| 0.445|  buying opportunity|[buying, opportun...|[buying, opportun...|[buying, opportun...|
| 0.661|Scaling Up on Lon...|[scaling, up, on,...|[scaling, long, p...|[scaling, long, p...|
|-0.763|its time to sell ...|[its, time, to, s...| [time, sell, banks]| [time, sell, banks]|
| 0.627|       Entering long|    [entering, long]|    [entering, long]|    [entering, long]|
| 0.653|      picked some up|  [picked, some, up]|            [picked]|            [picked]|
| 0.668|time to accumulat...|[time, to, accumu...|[time, accumulate...

In [15]:
"""
from functools import reduce
from pyspark.sql.functions import regexp_replace
source_df = sparkSession.createDataFrame(
    [
        ("h!o!c!k!e!y", "rangers", "new york"),
        ("soccer", "??nacional!!", "medellin")
    ],
    ["sport", "team", "city"]
)

print(source_df.show())
def remove_some_chars(col_name):
    removed_chars = ("!", "?")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, "")

actual_df = (reduce(
    lambda memo_df, col_name: memo_df.withColumn(col_name, remove_some_chars(col_name)),
    ["sport", "team"],
    source_df
))

print(actual_df.show())
"""

'\nfrom functools import reduce\nfrom pyspark.sql.functions import regexp_replace\nsource_df = sparkSession.createDataFrame(\n    [\n        ("h!o!c!k!e!y", "rangers", "new york"),\n        ("soccer", "??nacional!!", "medellin")\n    ],\n    ["sport", "team", "city"]\n)\n\nprint(source_df.show())\ndef remove_some_chars(col_name):\n    removed_chars = ("!", "?")\n    regexp = "|".join(\'\\{0}\'.format(i) for i in removed_chars)\n    return regexp_replace(col_name, regexp, "")\n\nactual_df = (reduce(\n    lambda memo_df, col_name: memo_df.withColumn(col_name, remove_some_chars(col_name)),\n    ["sport", "team"],\n    source_df\n))\n\nprint(actual_df.show())\n'

In [16]:
 """
 #now make term frequency vectors out of data frame to feed machine
from pyspark.ml.feature import HashingTF,IDF
hashingtf=HashingTF(inputCol="n-gram", outputCol="tf", numFeatures=100)
tf_df=hashingtf.transform(gram_df)
    """

'\n#now make term frequency vectors out of data frame to feed machine\nfrom pyspark.ml.feature import HashingTF,IDF\nhashingtf=HashingTF(inputCol="n-gram", outputCol="tf", numFeatures=100)\ntf_df=hashingtf.transform(gram_df)\n   '

In [17]:
"""
tf_df.select("tf").show(tf_df.select("tf").count())
"""

'\ntf_df.select("tf").show(tf_df.select("tf").count())\n'

In [18]:
"""
def square(x):
    return x**2
"""

'\ndef square(x):\n    return x**2\n'

In [27]:
#apply countvectorizer model
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import StringIndexer

# fit a CountVectorizerModel from the corpus.
cv = CountVectorizer(inputCol="n-gram", outputCol="CV_vector", minDF=1.0)

model = cv.fit(gram_df)

result = model.transform(gram_df)

In [28]:
#former le pipeline totale
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[tokenizer, remover, ngram,cv])
pipelineFit = pipeline.fit(spans)
final_df = pipelineFit.transform(spans)

In [29]:
#final_df=result.select("label","features").show(truncate=False)
final_df.show()

+------+--------------------+--------------------+--------------------+--------------------+--------------------+
| label|               spans|              tokens|            filtered|              n-gram|           CV_vector|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|-0.454|Putting on a litt...|[putting, on, a, ...|[putting, little,...|[putting, little,...|(44,[0,3,5,6],[1....|
|-0.464|          short some|       [short, some]|             [short]|             [short]|      (44,[0],[1.0])|
| 0.445|  buying opportunity|[buying, opportun...|[buying, opportun...|[buying, opportun...|(44,[4,18],[1.0,1...|
| 0.661|Scaling Up on Lon...|[scaling, up, on,...|[scaling, long, p...|[scaling, long, p...|(44,[1,7,26],[1.0...|
|-0.763|its time to sell ...|[its, time, to, s...| [time, sell, banks]| [time, sell, banks]|(44,[2,10,31],[1....|
| 0.627|       Entering long|    [entering, long]|    [entering, long]|    [entering, lo

In [30]:
"""
rdd=final_df.rdd
bow=rdd.map(lambda x: x[5]).take(final_df.count())
"""

'\nrdd=final_df.rdd\nbow=rdd.map(lambda x: x[5]).take(final_df.count())\n'

In [31]:
"""
final_df.select("features").take(5)
L=[]
for e in bow:
    L.append(e)
L[0][2]
"""

'\nfinal_df.select("features").take(5)\nL=[]\nfor e in bow:\n    L.append(e)\nL[0][2]\n'

In [32]:
"""
df = sparkSession.createDataFrame(
  [(0, ["a", "b", "c"]), (1, ["a", "b", "b", "c", "a"])],
  ["label", "raw"])
cv = CountVectorizer(inputCol="n-gram", outputCol="vectors")
model = cv.fit(data_ngram)
model.transform(data_ngram).show(truncate=False)
"""

'\ndf = sparkSession.createDataFrame(\n  [(0, ["a", "b", "c"]), (1, ["a", "b", "b", "c", "a"])],\n  ["label", "raw"])\ncv = CountVectorizer(inputCol="n-gram", outputCol="vectors")\nmodel = cv.fit(data_ngram)\nmodel.transform(data_ngram).show(truncate=False)\n'

In [33]:
from pyspark.mllib.linalg import Vectors
train_df=final_df.select("label","CV_vector")
#transformed_df = train_df.rdd.map(lambda row: LabeledPoint(row[0], Vectors.dense(row[1:])))

#X_train = train_df.rdd.map(lambda line:LabeledPoint(line[0],[line[1:]]))

In [34]:
from pyspark.ml.feature import VectorAssembler
features=["CV_vector"]
train_df.show(truncate=False)
vectorAssembler = VectorAssembler(inputCols=features, outputCol="features")  
transformed = vectorAssembler.transform(train_df)
transformed.printSchema()

+------+------------------------------------------------------+
|label |CV_vector                                             |
+------+------------------------------------------------------+
|-0.454|(44,[0,3,5,6],[1.0,1.0,1.0,1.0])                      |
|-0.464|(44,[0],[1.0])                                        |
|0.445 |(44,[4,18],[1.0,1.0])                                 |
|0.661 |(44,[1,7,26],[1.0,1.0,1.0])                           |
|-0.763|(44,[2,10,31],[1.0,1.0,1.0])                          |
|0.627 |(44,[1,21],[1.0,1.0])                                 |
|0.653 |(44,[28],[1.0])                                       |
|0.668 |(44,[1,2,7,19,33,34,40],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|0.46  |(44,[12,25,29,30,36,38],[1.0,1.0,1.0,1.0,1.0,1.0])    |
|0.403 |(44,[8,15,32,42],[1.0,1.0,1.0,1.0])                   |
|0.0   |(44,[9,11,13,35,39,43],[1.0,1.0,1.0,1.0,1.0,1.0])     |
|-0.438|(44,[20,23,37],[1.0,1.0,1.0])                         |
|-0.398|(44,[16,17,24],[1.0,1.0,1.0])   

In [35]:
from pyspark.ml.feature import *
vhouse_df = transformed.select(['features', 'label'])
(training, test) = vhouse_df.randomSplit([.7, .3])
#X_train = training.rdd.map(lambda line:LabeledPoint(line[0],[line[1:]]))

In [36]:
vhouse_df.show()

+--------------------+------+
|            features| label|
+--------------------+------+
|(44,[0,3,5,6],[1....|-0.454|
|      (44,[0],[1.0])|-0.464|
|(44,[4,18],[1.0,1...| 0.445|
|(44,[1,7,26],[1.0...| 0.661|
|(44,[2,10,31],[1....|-0.763|
|(44,[1,21],[1.0,1...| 0.627|
|     (44,[28],[1.0])| 0.653|
|(44,[1,2,7,19,33,...| 0.668|
|(44,[12,25,29,30,...|  0.46|
|(44,[8,15,32,42],...| 0.403|
|(44,[9,11,13,35,3...|   0.0|
|(44,[20,23,37],[1...|-0.438|
|(44,[16,17,24],[1...|-0.398|
|(44,[4,22,41],[1....| 0.483|
|(44,[0,14,27],[1....| -0.48|
|(44,[0,3,5,6],[1....|-0.454|
+--------------------+------+



In [37]:
"""
from pyspark.ml.regression import RandomForestRegressor
#model = RandomForest.trainRegressor(X_train,{}, numTrees =10,maxDepth =None,maxBins =32, seed=42)
RF=RandomForestRegressor(labelCol='label', featuresCol='features',numTrees=2, maxDepth=2, seed=42)
model = RF.fit(training)
"""

"\nfrom pyspark.ml.regression import RandomForestRegressor\n#model = RandomForest.trainRegressor(X_train,{}, numTrees =10,maxDepth =None,maxBins =32, seed=42)\nRF=RandomForestRegressor(labelCol='label', featuresCol='features',numTrees=2, maxDepth=2, seed=42)\nmodel = RF.fit(training)\n"

In [46]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor

#model = LinearRegression(featuresCol = 'features', labelCol='label', maxIter=10, regParam=0.3, elasticNetParam=0.8)
model = RandomForestRegressor(featuresCol = 'features', labelCol='label',numTrees=2, maxDepth=2, seed=42)
#lr_model = model.fit(training)
rf_model = model.fit(training)
#print("Coefficients: " + str(rf_model.coefficients))
#print("Intercept: " + str(rf_model.intercept))

In [48]:
"""
#----------------------LinearRegression---------------------
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
"""

'\n#----------------------LinearRegression---------------------\ntrainingSummary = lr_model.summary\nprint("RMSE: %f" % trainingSummary.rootMeanSquaredError)\nprint("r2: %f" % trainingSummary.r2)\n'

In [47]:
#-----------------------RandomForest-----------------
rf_predictions = rf_model.transform(test)
rf_predictions.select("prediction","label","features").show(5)

+-------------------+------+--------------------+
|         prediction| label|            features|
+-------------------+------+--------------------+
|-0.2477916666666667|-0.454|(44,[0,3,5,6],[1....|
|-0.2477916666666667| 0.403|(44,[8,15,32,42],...|
|-0.2477916666666667|-0.398|(44,[16,17,24],[1...|
+-------------------+------+--------------------+



In [40]:
"""
rdd = sc.textFile("text.txt")
rdd.take(5)
def f(lines):
    lines = lines.lower()
    lines = lines.split(" ")
    return lines
rdd1 = rdd.map(f)
"""

'\nrdd = sc.textFile("text.txt")\nrdd.take(5)\ndef f(lines):\n    lines = lines.lower()\n    lines = lines.split(" ")\n    return lines\nrdd1 = rdd.map(f)\n'

In [41]:
"""
label=df.map(lambda x: x[2]).take(5)
tweet=df.map(lambda x: x[4]).take(5)
print(label[0])
data=[]
i=0
for e in label:
    data.append(LabeledPoint(label[i], tweet[i]))
    i=i+1
"""

'\nlabel=df.map(lambda x: x[2]).take(5)\ntweet=df.map(lambda x: x[4]).take(5)\nprint(label[0])\ndata=[]\ni=0\nfor e in label:\n    data.append(LabeledPoint(label[i], tweet[i]))\n    i=i+1\n'

In [42]:
"""
#make a labeled point from a DF
l2=df.map(lambda row: LabeledPoint(row.label, row.features))
#pos = LabeledPoint(label, tweet)
data = [
     LabeledPoint(0.0, [0.0]),
     LabeledPoint(0.0, [1.0]),
     LabeledPoint(1.0, [2.0]),
     LabeledPoint(1.0, [3.0])
 ]
data.append(LabeledPoint(1.0, [3.0]))
type(data)
"""

'\n#make a labeled point from a DF\nl2=df.map(lambda row: LabeledPoint(row.label, row.features))\n#pos = LabeledPoint(label, tweet)\ndata = [\n     LabeledPoint(0.0, [0.0]),\n     LabeledPoint(0.0, [1.0]),\n     LabeledPoint(1.0, [2.0]),\n     LabeledPoint(1.0, [3.0])\n ]\ndata.append(LabeledPoint(1.0, [3.0]))\ntype(data)\n'